In [41]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [42]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset
import torch
import pandas as pd

In [43]:
df = pd.read_csv("Iteration__1.csv")
dataset = Dataset.from_pandas(df)
# Load your dataset
data = load_dataset("csv", data_files={"train": dataset_path})

In [44]:
train_test_split = dataset.train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [45]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/apps/jupyter/6.5.4/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [46]:
# label_to_id = {"HOUSING": 0, "TRANSPORTATION": 1, "other_labels": 2}  # Add all your labels accordingly
# id_to_label = {v: k for k, v in label_to_id.items()}

In [47]:
def preprocess_data(examples):
    # Prefix to guide the model on what task to perform
    prefix = "Classify SDOH and adversity: "
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["label"], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [56]:
# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_data, batched=True, )


Map: 100%|██████████| 180/180 [00:00<00:00, 2236.24 examples/s]


{'text': 'Patient arrived with her fiancé, concerned about a recent change in bowel habits',
 'label': 'RELATIONSHIP',
 'adverse': 'nonadverse',
 'input_ids': [4501,
  4921,
  9579,
  9195,
  11,
  3,
  9,
  26,
  2660,
  485,
  10,
  17656,
  4363,
  28,
  160,
  361,
  152,
  75,
  154,
  6,
  4376,
  81,
  3,
  9,
  1100,
  483,
  16,
  3,
  27169,
  8966,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [49]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("t5-base")

training_args = TrainingArguments(
    output_dir="./sdoh_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()



Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/apps/jupyter/6.5.4/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.081076
2,No log,0.076731
3,No log,0.069663


TrainOutput(global_step=180, training_loss=2.5269376966688366, metrics={'train_runtime': 73.9694, 'train_samples_per_second': 29.201, 'train_steps_per_second': 2.433, 'total_flos': 1315349043609600.0, 'train_loss': 2.5269376966688366, 'epoch': 3.0})

In [50]:
trainer.save_model("./sdoh_t5_model")
tokenizer.save_pretrained("./sdoh_t5_model")


('./sdoh_t5_model/tokenizer_config.json',
 './sdoh_t5_model/special_tokens_map.json',
 './sdoh_t5_model/spiece.model',
 './sdoh_t5_model/added_tokens.json')

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_texts"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([id_to_label[label] for label in examples["labels"]], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = tokenized_data.map(tokenize_function, batched=True)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./sdoh_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    # Ensure you have an evaluation dataset or remove the eval_dataset argument
    # eval_dataset=tokenized_datasets["test"],
)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("./sdoh_finetuned_model")
tokenizer.save_pretrained("./sdoh_finetuned_model")

In [ ]:
# # Define a function to preprocess your data
# def preprocess_data(examples):
#     # Adjust these lines to match your dataset structure
#     input_text = "classify: " + examples["text"]
#     labels = examples["label"]  # Ensure your labels are appropriately encoded as integers or strings
#     return {"input_texts": input_text, "labels": labels}

# # Load your dataset (replace 'path_to_your_dataset' with the actual path or dataset loading method)
# data = load_dataset('path_to_your_dataset')

# # Process your dataset
# tokenized_data = data.map(preprocess_data)

# # Load the tokenizer and model
# model_name = "t5-base"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

# # Tokenize inputs and labels
# def tokenize_function(examples):
#     model_inputs = tokenizer(examples["input_texts"], padding="max_length", truncation=True)
#     # Tokenize the labels as well
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples["labels"], padding="max_length", truncation=True)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_datasets = tokenized_data.map(tokenize_function, batched=True)

# # Load the tokenized data into a DataLoader
# train_dataset = tokenized_datasets["train"]
# eval_dataset = tokenized_datasets["test"]

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

# # Initialize Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
# )

# # Train the model
# trainer.train()